In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import itertools
from copy import deepcopy
from collections import defaultdict
import os, re, ast
import math
from itertools import cycle
import time

In [ ]:
# Read dataset "books.csv" and assign it to the variable b
b = pd.read_csv('books.csv')
print(b.shape)
b.head()

(2312, 5)


,book_id,avg_rating,no_of_ratings,user_id,user_rating
0,4833,4.25,7156.0,3466,0
1,590,4.31,7821.0,3466,5
2,4264,4.08,3836.0,3453,5
3,3361,3.52,1245.0,3453,4
4,4535,4.13,3107.0,3453,0


In [ ]:
# As an example we check which users rated book with id = 1041
# Here we have two assumption:
#   First one is that:  "if a user gave a rating to some book it means that user bought that book in past"
# As you see, there are three index numbers for this dataset 730, 1108 and 2055. 
#   Second assumption is:  "first user with id=1283 bought this book and then 
#                           user with id=2215 bought this book and then user with id=3469 bought this book"
# Therefore we think the order of indexes as order of transactions. 
# We can continue our work only after having such assumptions
b[b['book_id'] == 1041]

,book_id,avg_rating,no_of_ratings,user_id,user_rating
730,1041,4.0,4194.0,1283,3
1108,1041,4.0,4194.0,2215,4
2055,1041,4.0,4194.0,3469,4


In [ ]:
# We don't want to recommend book which it's rating is less than 3
# Therefore we drop such books from our dataset
tmp = b[b['user_rating'] >= 3]
tmp.head()

,book_id,avg_rating,no_of_ratings,user_id,user_rating
1,590,4.31,7821.0,3466,5
2,4264,4.08,3836.0,3453,5
3,3361,3.52,1245.0,3453,4
5,518,4.08,6250.0,3466,4
9,225,4.09,3957.0,3453,5


In [ ]:
# We drop following columns because we don't need them in our implementation
tmp.drop(['avg_rating', 'no_of_ratings', 'user_rating'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
tmp.head()

,book_id,user_id
1,590,3466
2,4264,3453
3,3361,3453
5,518,3466
9,225,3453


In [ ]:
# As you noticed size of our dataset srhinked from 2312 to 1302
tmp.shape

(1302, 2)

In [ ]:
# We apply groupby function on the dataset to group events by their users
# that's it is necessary to know that which user bought which books in our implementation 
df = tmp.groupby('user_id').book_id.apply(list).reset_index()

In [ ]:
# Each list represents a set of books that were bought by one user
# We also drop lists with length less than 2 because such sequences wouldn't be
# useful in our implementation based on sequential data mining algorithms' needs
for i in range(df.shape[0]):
    print(df.book_id[i])
    if len(df.book_id[i]) < 2:
      df.drop([i], axis='index', inplace=True)

[4373, 323, 907, 4632, 4576, 645, 538, 208, 4360, 1765, 4035, 4978, 4830, 1275, 4457, 444, 3732, 4889, 90]
[2396, 674, 365, 47, 1790, 379, 3926, 3723, 1453, 477, 3073, 1616, 784, 4058, 3930, 838]
[2967]
[993, 4208, 3744, 3840, 379, 784, 3930, 3329, 1545]
[3311]
[3702]
[716]
[3196, 61, 3769, 4540, 3389, 3643, 1884, 3609, 4278, 1393]
[376]
[639]
[4041, 652]
[851]
[1722]
[9]
[2096]
[3720]
[3875]
[540, 4968, 1583, 2089, 4667, 766, 1127, 4203, 1577, 2984, 3926, 477, 1616, 838, 1826, 4035, 1275, 1257, 444, 4995, 690]
[1062]
[3367, 149, 2077, 3731]
[2689]
[1417]
[21]
[4288]
[1676]
[2913]
[4122]
[1824]
[1613, 1667]
[2629]
[2078]
[1092, 4185]
[4854]
[2266, 1041, 3609, 2295, 2637, 4004, 3815, 3214, 2605, 3677, 200, 1759, 4696, 1773]
[3137, 3785, 3798, 716, 3169, 2062, 2016, 4853, 1307, 2166, 209, 2133, 3090, 1716, 2519, 906, 4334, 3014, 1861, 3367]
[1509]
[1580]
[3781]
[1613]
[4422]
[298, 4827]
[3050]
[3082]
[1894, 4508]
[3835]
[2062]
[3393]
[1334, 4885, 3221, 3974]
[927]
[968]
[1216]
[754]
[439

In [ ]:
# You can see previous results in better form here
# You can easily realize which users bought which books
df

,user_id,book_id
0,117,"[4373, 323, 907, 4632, 4576, 645, 538, 208, 43..."
1,176,"[2396, 674, 365, 47, 1790, 379, 3926, 3723, 14..."
3,295,"[993, 4208, 3744, 3840, 379, 784, 3930, 3329, ..."
7,397,"[3196, 61, 3769, 4540, 3389, 3643, 1884, 3609,..."
10,484,"[4041, 652]"
...,...,...
134,3483,"[3071, 2078, 9, 1995, 2240, 3311, 4021, 4755, ..."
135,3484,"[2339, 4437, 1448, 414, 535, 4390, 3720, 3302,..."
136,3486,"[2351, 3004]"
151,7130,"[4901, 4601, 2869, 2252]"


In [ ]:
# We think each book list as a sequence for our data mining algorithm
sequences = []
for i in range(df.shape[0]):
  sequences.append(df.iloc[i]['book_id'])


In [ ]:
# And we took each event from the sequence and convert it to string
# and then put it another list
events_lists = []
for sequence in sequences:
  temp = []
  for event in sequence:
   temp.append([str(event)])
  events_lists.append(temp)

In [ ]:
for i in sequences:
  print(i)

[4373, 323, 907, 4632, 4576, 645, 538, 208, 4360, 1765, 4035, 4978, 4830, 1275, 4457, 444, 3732, 4889, 90]
[2396, 674, 365, 47, 1790, 379, 3926, 3723, 1453, 477, 3073, 1616, 784, 4058, 3930, 838]
[993, 4208, 3744, 3840, 379, 784, 3930, 3329, 1545]
[3196, 61, 3769, 4540, 3389, 3643, 1884, 3609, 4278, 1393]
[4041, 652]
[540, 4968, 1583, 2089, 4667, 766, 1127, 4203, 1577, 2984, 3926, 477, 1616, 838, 1826, 4035, 1275, 1257, 444, 4995, 690]
[3367, 149, 2077, 3731]
[1613, 1667]
[1092, 4185]
[2266, 1041, 3609, 2295, 2637, 4004, 3815, 3214, 2605, 3677, 200, 1759, 4696, 1773]
[3137, 3785, 3798, 716, 3169, 2062, 2016, 4853, 1307, 2166, 209, 2133, 3090, 1716, 2519, 906, 4334, 3014, 1861, 3367]
[298, 4827]
[1894, 4508]
[1334, 4885, 3221, 3974]
[2260, 3733]
[3319, 3351]
[4941, 2936, 2878, 1476, 3090, 2816, 669, 3265, 4247, 29]
[4868, 2491, 4854, 1209, 1580, 74, 1041, 2924, 2936, 2638, 1476, 4432, 593, 2236, 241, 1685, 4682, 2718, 3500]
[81, 647]
[1209, 1613, 3798, 1062, 3169, 2062, 541, 21, 2853, 1

In [ ]:
# Sequences are same as before, but we changed their representation in this way
# The reason for such preprocessing logic is that we implemented GSP mining algorithm
# in data mining course from scratch  and it accepts inputs only as the below format
for k in events_lists:
  print(k)

[['4373'], ['323'], ['907'], ['4632'], ['4576'], ['645'], ['538'], ['208'], ['4360'], ['1765'], ['4035'], ['4978'], ['4830'], ['1275'], ['4457'], ['444'], ['3732'], ['4889'], ['90']]
[['2396'], ['674'], ['365'], ['47'], ['1790'], ['379'], ['3926'], ['3723'], ['1453'], ['477'], ['3073'], ['1616'], ['784'], ['4058'], ['3930'], ['838']]
[['993'], ['4208'], ['3744'], ['3840'], ['379'], ['784'], ['3930'], ['3329'], ['1545']]
[['3196'], ['61'], ['3769'], ['4540'], ['3389'], ['3643'], ['1884'], ['3609'], ['4278'], ['1393']]
[['4041'], ['652']]
[['540'], ['4968'], ['1583'], ['2089'], ['4667'], ['766'], ['1127'], ['4203'], ['1577'], ['2984'], ['3926'], ['477'], ['1616'], ['838'], ['1826'], ['4035'], ['1275'], ['1257'], ['444'], ['4995'], ['690']]
[['3367'], ['149'], ['2077'], ['3731']]
[['1613'], ['1667']]
[['1092'], ['4185']]
[['2266'], ['1041'], ['3609'], ['2295'], ['2637'], ['4004'], ['3815'], ['3214'], ['2605'], ['3677'], ['200'], ['1759'], ['4696'], ['1773']]
[['3137'], ['3785'], ['3798'],

# Our implementation

In [ ]:
# You can change minimum support (minsup)  if you want
# Algorithm starts  from this cell number 53 and will end in cell number 64
C = {}
F = {}
S = {}
minsup = 4

In [ ]:
# Created Candidate 1 itemset
# Creation of C_1
C_1 = defaultdict(int)
for events_row in events_lists:
  for event in np.unique(events_row):
    if event in C_1:
      C_1[event] += 1
    else:
      C_1[event] = 1

In [ ]:
C_1

defaultdict(int,
            {'1006': 1,
             '1031': 2,
             '1036': 1,
             '104': 2,
             '1041': 3,
             '1061': 1,
             '1062': 1,
             '107': 2,
             '1089': 2,
             '1091': 2,
             '1092': 3,
             '110': 3,
             '1127': 1,
             '1128': 2,
             '1143': 1,
             '1155': 1,
             '1180': 1,
             '1186': 3,
             '119': 2,
             '1191': 2,
             '1192': 2,
             '1195': 1,
             '1197': 3,
             '1209': 3,
             '1210': 2,
             '1216': 1,
             '1217': 2,
             '1223': 4,
             '1231': 2,
             '1242': 1,
             '1249': 3,
             '1250': 1,
             '1257': 1,
             '1264': 2,
             '1275': 2,
             '1292': 2,
             '1293': 2,
             '130': 1,
             '1300': 1,
             '1302': 3,
             '1307': 1,
    

In [ ]:
# Create Frequet-1 subsequence
# Creation of F_1
F_1 = {}
for item in C_1:
  if C_1[item] >= minsup:
    F_1[item] = C_1[item]

In [ ]:
F_1 

{'1223': 4,
 '144': 4,
 '149': 4,
 '1490': 4,
 '1545': 4,
 '2077': 5,
 '2181': 4,
 '2519': 4,
 '2605': 5,
 '2618': 4,
 '2637': 4,
 '2770': 4,
 '278': 4,
 '298': 4,
 '3014': 4,
 '3207': 5,
 '3214': 5,
 '336': 4,
 '3367': 4,
 '354': 4,
 '3693': 4,
 '3938': 4,
 '3951': 4,
 '4031': 4,
 '4062': 4,
 '412': 4,
 '4183': 4,
 '4185': 4,
 '4330': 4,
 '4437': 4,
 '444': 4,
 '4611': 4,
 '4673': 4,
 '4696': 4,
 '4707': 5,
 '478': 4,
 '4902': 4,
 '4941': 4,
 '907': 4}

In [ ]:
C['C_1'] = C_1
S['S_1'] = list(F_1.keys())

In [ ]:
with open('output.txt', 'w') as f:
  for i in S['S_1']:
    f.write("('" + str(i) + "')")
    f.write('\n')


In [ ]:
def findLen(S):
  count = 0
  for i in S:
    count += 1
  return count

In [ ]:
# Generate Length-2 Candidates
C_candidate = []
for subset in itertools.permutations(S['S_1'], 2):
  C_candidate.append(subset)
for i in S['S_1']:
  k = list()
  k.append(i)
  k.append(i)
  C_candidate.append(tuple(k))
C['C_2'] = C_candidate

In [ ]:
S['S_1']

['444',
 '907',
 '1545',
 '149',
 '2077',
 '3367',
 '4185',
 '2605',
 '2637',
 '3214',
 '4696',
 '2519',
 '3014',
 '298',
 '4941',
 '1223',
 '2181',
 '3207',
 '354',
 '4062',
 '4673',
 '412',
 '4611',
 '336',
 '3693',
 '4707',
 '478',
 '4330',
 '278',
 '4183',
 '144',
 '2770',
 '3951',
 '4902',
 '2618',
 '4031',
 '3938',
 '4437',
 '1490']

In [ ]:
C['C_2']

[('444', '907'),
 ('444', '1545'),
 ('444', '149'),
 ('444', '2077'),
 ('444', '3367'),
 ('444', '4185'),
 ('444', '2605'),
 ('444', '2637'),
 ('444', '3214'),
 ('444', '4696'),
 ('444', '2519'),
 ('444', '3014'),
 ('444', '298'),
 ('444', '4941'),
 ('444', '1223'),
 ('444', '2181'),
 ('444', '3207'),
 ('444', '354'),
 ('444', '4062'),
 ('444', '4673'),
 ('444', '412'),
 ('444', '4611'),
 ('444', '336'),
 ('444', '3693'),
 ('444', '4707'),
 ('444', '478'),
 ('444', '4330'),
 ('444', '278'),
 ('444', '4183'),
 ('444', '144'),
 ('444', '2770'),
 ('444', '3951'),
 ('444', '4902'),
 ('444', '2618'),
 ('444', '4031'),
 ('444', '3938'),
 ('444', '4437'),
 ('444', '1490'),
 ('907', '444'),
 ('907', '1545'),
 ('907', '149'),
 ('907', '2077'),
 ('907', '3367'),
 ('907', '4185'),
 ('907', '2605'),
 ('907', '2637'),
 ('907', '3214'),
 ('907', '4696'),
 ('907', '2519'),
 ('907', '3014'),
 ('907', '298'),
 ('907', '4941'),
 ('907', '1223'),
 ('907', '2181'),
 ('907', '3207'),
 ('907', '354'),
 ('90

In [ ]:

with open("output.txt", 'a') as f:

  len = 2
  while findLen(S['S_'+str(len-1)]) !=  0:
    print('INSIDE LOOP', len,'-length Sequence \n')

    if len > 2:
      # CANDIDATE GENERATION
      C['C_'+str(len)] = []
      for i in range(findLen(S['S_'+str(len-1)])):
        for j in range(0, findLen(S['S_'+str(len-1)])):
          if S['S_'+str(len-1)][i][1:] == S['S_'+str(len-1)][j][:-1]:
            C['C_'+str(len)] .append(S['S_'+str(len-1)][i] + S['S_'+str(len-1)][j][len-2:])

      # Pruning
      C_candidate = []
      for i in range(findLen(C['C_'+str(len)] )):
        temp = []
        for subset in itertools.combinations(C['C_'+str(len)][i], len-1):
          temp.append(subset)
        count = 0
        for event in temp:
          if event in S['S_'+str(len-1)]:
            count += 1
        if count == len:
          C_candidate.append(C['C_'+str(len)][i])

    # Support Counting
    C_cand_sup = {}
    for c_event in C_candidate:  
      #print('c_event: ', c_event)
      for events_row in events_lists:
        licycle = cycle(c_event)
        item = next(licycle)
        count=0
        for event in events_row:
          if item in event:
            count += 1
            if count  == len:
              licycle = cycle(c_event)
              item = next(licycle)
              if c_event in C_cand_sup:
                C_cand_sup[c_event] += 1
              else:
                C_cand_sup[c_event] = 1
              #print(c_event, ' is in ', events_row)
              break
            item = next(licycle)
            #print('Item: ', item)  

    # CANDIDATE ELIMINATION
    S['S_'+str(len)] = {}
    for event in C_cand_sup:
      if C_cand_sup[event] >= minsup:
        S['S_'+str(len)][event] = C_cand_sup[event]

    print(S['S_'+str(len)])
    s = ''
    for i in S['S_'+str(len)]:
      s = str(i)
      f.write(str(s))
      f.write('\n')
    S['S_'+str(len)] = list(S['S_'+str(len)].keys())

    len += 1
    


INSIDE LOOP 2 -length Sequence 

{('149', '2077'): 4, ('3214', '2605'): 5, ('4673', '1223'): 4, ('4707', '478'): 4, ('3951', '144'): 4, ('3951', '4902'): 4, ('4902', '144'): 4}
INSIDE LOOP 3 -length Sequence 

{('3951', '4902', '144'): 4}
INSIDE LOOP 4 -length Sequence 

{}


## To see result of the algorithm you can open the file "output.txt" which is produced by our program. That file contains all sequential pattern for the given minimum support value.

We would say that the result is not satisfiable from our point of view because there are length 2 sequence and only one length four sequence for minimum suport 4. We think that we wouldn't be successful in our approach. Also minum support 4 is much less for our mining algorithm and the patterns also very poor. Maybe we have problems in our assumptions. We think that the main reason is our dataset. 